In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip /content/drive/MyDrive/Applied_AI_project/dataset_1.zip

Archive:  /content/drive/MyDrive/Applied_AI_project/dataset_1.zip
   creating: dataset_1/bacterial spot/
  inflating: dataset_1/bacterial spot/Bs1.JPG  
  inflating: dataset_1/bacterial spot/Bs1_change_180.jpg  
  inflating: dataset_1/bacterial spot/Bs1_change_90.jpg  
  inflating: dataset_1/bacterial spot/Bs1_hight.jpg  
  inflating: dataset_1/bacterial spot/Bs1_lower.jpg  
  inflating: dataset_1/bacterial spot/Bs1_mirror.jpg  
  inflating: dataset_1/bacterial spot/Bs1_mirror_vertical.jpg  
  inflating: dataset_1/bacterial spot/Bs10.JPG  
  inflating: dataset_1/bacterial spot/Bs10_change_180.jpg  
  inflating: dataset_1/bacterial spot/Bs10_change_270.jpg  
  inflating: dataset_1/bacterial spot/Bs10_change_90.jpg  
  inflating: dataset_1/bacterial spot/Bs10_hight.jpg  
  inflating: dataset_1/bacterial spot/Bs10_lower.jpg  
  inflating: dataset_1/bacterial spot/Bs10_mirror.jpg  
  inflating: dataset_1/bacterial spot/Bs10_mirror_vertical.jpg  
  inflating: dataset_1/bacterial spot/Bs100.

In [3]:
DATASET_TYPE = 'dataset_1'

In [4]:
import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss
from torch.optim import Adam

import torchvision
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18, mobilenet_v2, vgg16

import time

In [5]:
# splitting the dataset into specific folders for train, validate and test
%pip install split-folders

import splitfolders

source_folder = f'{DATASET_TYPE}'
target_folder = f'{DATASET_TYPE}_splitted'

train_ratio = .7
val_ratio = .15
test_ratio = .15

splitfolders.ratio(source_folder, output=target_folder, seed=1337, ratio=(train_ratio, val_ratio, test_ratio), group_prefix=None, move=False)

Copying files: 2009 files [00:00, 4900.32 files/s]


In [6]:
# applying data prepocessing and loading dataset
train_dataset_tranform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_dataset_tranform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = ImageFolder(f"{DATASET_TYPE}_splitted/train", transform=train_dataset_tranform)
train_length = len(train_dataset)

validate_dataset = ImageFolder(f"{DATASET_TYPE}_splitted/val", transform=test_dataset_tranform)
validate_length = len(validate_dataset)

test_dataset = ImageFolder(f"{DATASET_TYPE}_splitted/test", transform=test_dataset_tranform)
test_dataset_length = len(test_dataset)

dataset_classes = train_dataset.classes
print(dataset_classes)

['bacterial spot', 'healthy', 'late blight']


In [7]:
batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validate_loader = DataLoader(validate_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [14]:
# setting up the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [8]:
# modifying architecture output classes
class ResNet18(nn.Module):
  def __init__(self, num_of_classes):
    super(ResNet18, self).__init__()
    self.model = resnet18(weights=None)
    features = self.model.fc.in_features
    self.model.fc = nn.Linear(features, num_of_classes)

  def forward(self, x):
    x = self.model(x)
    return x

In [9]:
# modifying architecture output classes
class MobileNetV2(nn.Module):
  def __init__(self, num_of_classes):
    super(MobileNetV2, self).__init__()
    self.model = mobilenet_v2(weights=None)
    features = self.model.classifier[-1].in_features
    self.model.classifier[-1] = nn.Linear(features, num_of_classes)

  def forward(self, x):
    x = self.model(x)
    return x

In [10]:
# modifying architecture output classes
class VGG16(nn.Module):
  def __init__(self, num_of_classes):
    super(VGG16, self).__init__()
    self.model = vgg16(weights=None)
    features = self.model.classifier[-1].in_features
    self.model.classifier[-1] = nn.Linear(features, num_of_classes)

  def forward(self, x):
    x = self.model(x)
    return x

In [11]:
# defining model, optimizer, architecture
learning_rate = 0.001
epochs = 1

In [15]:
model_1 = ResNet18(num_of_classes=len(dataset_classes))
model_1 = model_1.to(device)
optimizer = Adam(model_1.parameters(), lr=learning_rate)
loss_function = CrossEntropyLoss()

print(model_1)

ResNet18(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runn

In [16]:
train_losses = []
train_accuracies = []
validate_losses = []
validate_accuracies = []

start_time = time.time()

for epoch in range(epochs):

  print(f"epoch number ---> {epoch + 1} ")

  model_1.train()
  train_batch_loss = 0.0
  train_correct_predictions = 0
  train_samples = 0
  for xtrain, ytrain in train_loader:

    optimizer.zero_grad()

    xtrain = xtrain.to(device)
    ytrain = ytrain.to(device)
    train_prob = model_1(xtrain)

    # calculating batch accuracy
    _, train_prediction = torch.max(train_prob, 1)
    train_samples += ytrain.size(0)
    train_correct_predictions += (train_prediction == ytrain).sum().item()

    loss = loss_function(train_prob, ytrain)
    loss.backward()
    optimizer.step()

    # calculating batch loss
    train_batch_loss += loss.item()

  # calculating train loss for epoch
  train_loss = train_batch_loss / len(train_loader)
  train_losses.append(train_loss)

  # calculating train accuracy for epoch
  train_accuracy = 100 * (train_correct_predictions / train_samples)
  train_accuracies.append(train_accuracy)
  print("training loss: {:.4f}  training accuracy: {:.2f} %".format(train_loss, train_accuracy))

  validate_batch_loss = 0.0
  validate_correct_predictions = 0
  validate_samples = 0
  model_1.eval()
  with torch.no_grad():
    for xvalidate, yvalidate in validate_loader:

      optimizer.zero_grad()

      xvalidate = xvalidate.to(device)
      yvalidate = yvalidate.to(device)
      validate_prob = model_1(xvalidate)

      #calculating batch accuracy
      _, validate_prediction = torch.max(validate_prob, 1)
      validate_samples += yvalidate.size(0)
      validate_correct_predictions += (validate_prediction == yvalidate).sum().item()

      loss = loss_function(validate_prob, yvalidate)

      #calculating batch loss
      validate_batch_loss += loss.item()

  # calculating validate loss for epoch
  validate_loss = validate_batch_loss / len(validate_loader)
  validate_losses.append(validate_loss)

  # calculating validate accuracy for epoch
  validate_accuracy = 100 * (validate_correct_predictions / validate_samples)
  validate_accuracies.append(validate_accuracy)
  print("validation loss: {:.4f}  validation accuracy: {:.2f} %".format(validate_loss, validate_accuracy))

end_time = time.time()
epoch_time = end_time - start_time

print("Wall Clock Time for One Epoch RestNet18:", epoch_time, "seconds")

epoch number ---> 1 
training loss: 0.9899  training accuracy: 57.76 %
validation loss: 1.1268  validation accuracy: 57.67 %
Wall Clock Time for One Epoch RestNet18: 20.239220142364502 seconds


In [18]:
model_2 = MobileNetV2(num_of_classes=len(dataset_classes))
model_2 = model_2.to(device)
optimizer = Adam(model_2.parameters(), lr=learning_rate)
loss_function = CrossEntropyLoss()

print(model_2)

MobileNetV2(
  (model): MobileNetV2(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU6(inplace=True)
          )
          (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (2): InvertedResidual(
        (conv): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(16, 96, kernel_size=(1, 

In [19]:
train_losses = []
train_accuracies = []
validate_losses = []
validate_accuracies = []

start_time = time.time()

for epoch in range(epochs):

  print(f"epoch number ---> {epoch + 1} ")

  model_2.train()
  train_batch_loss = 0.0
  train_correct_predictions = 0
  train_samples = 0
  for xtrain, ytrain in train_loader:

    optimizer.zero_grad()

    xtrain = xtrain.to(device)
    ytrain = ytrain.to(device)
    train_prob = model_2(xtrain)

    # calculating batch accuracy
    _, train_prediction = torch.max(train_prob, 1)
    train_samples += ytrain.size(0)
    train_correct_predictions += (train_prediction == ytrain).sum().item()

    loss = loss_function(train_prob, ytrain)
    loss.backward()
    optimizer.step()

    # calculating batch loss
    train_batch_loss += loss.item()

  # calculating train loss for epoch
  train_loss = train_batch_loss / len(train_loader)
  train_losses.append(train_loss)

  # calculating train accuracy for epoch
  train_accuracy = 100 * (train_correct_predictions / train_samples)
  train_accuracies.append(train_accuracy)
  print("training loss: {:.4f}  training accuracy: {:.2f} %".format(train_loss, train_accuracy))

  validate_batch_loss = 0.0
  validate_correct_predictions = 0
  validate_samples = 0
  model_2.eval()
  with torch.no_grad():
    for xvalidate, yvalidate in validate_loader:

      optimizer.zero_grad()

      xvalidate = xvalidate.to(device)
      yvalidate = yvalidate.to(device)
      validate_prob = model_2(xvalidate)

      #calculating batch accuracy
      _, validate_prediction = torch.max(validate_prob, 1)
      validate_samples += yvalidate.size(0)
      validate_correct_predictions += (validate_prediction == yvalidate).sum().item()

      loss = loss_function(validate_prob, yvalidate)

      #calculating batch loss
      validate_batch_loss += loss.item()

  # calculating validate loss for epoch
  validate_loss = validate_batch_loss / len(validate_loader)
  validate_losses.append(validate_loss)

  # calculating validate accuracy for epoch
  validate_accuracy = 100 * (validate_correct_predictions / validate_samples)
  validate_accuracies.append(validate_accuracy)
  print("validation loss: {:.4f}  validation accuracy: {:.2f} %".format(validate_loss, validate_accuracy))

end_time = time.time()
epoch_time = end_time - start_time

print("Wall Clock Time for One Epoch MobileNetV2:", epoch_time, "seconds")

epoch number ---> 1 
training loss: 0.9070  training accuracy: 53.28 %
validation loss: 1.7064  validation accuracy: 35.00 %
Wall Clock Time for One Epoch MobileNetV2: 18.982508182525635 seconds


In [20]:
model_3 = VGG16(num_of_classes=len(dataset_classes))
model_3 = model_3.to(device)
optimizer = Adam(model_3.parameters(), lr=learning_rate)
loss_function = CrossEntropyLoss()

print(model_3)

VGG16(
  (model): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (16): Ma

In [21]:
train_losses = []
train_accuracies = []
validate_losses = []
validate_accuracies = []

start_time = time.time()

for epoch in range(epochs):

  print(f"epoch number ---> {epoch + 1} ")

  model_3.train()
  train_batch_loss = 0.0
  train_correct_predictions = 0
  train_samples = 0
  for xtrain, ytrain in train_loader:

    optimizer.zero_grad()

    xtrain = xtrain.to(device)
    ytrain = ytrain.to(device)
    train_prob = model_3(xtrain)

    # calculating batch accuracy
    _, train_prediction = torch.max(train_prob, 1)
    train_samples += ytrain.size(0)
    train_correct_predictions += (train_prediction == ytrain).sum().item()

    loss = loss_function(train_prob, ytrain)
    loss.backward()
    optimizer.step()

    # calculating batch loss
    train_batch_loss += loss.item()

  # calculating train loss for epoch
  train_loss = train_batch_loss / len(train_loader)
  train_losses.append(train_loss)

  # calculating train accuracy for epoch
  train_accuracy = 100 * (train_correct_predictions / train_samples)
  train_accuracies.append(train_accuracy)
  print("training loss: {:.4f}  training accuracy: {:.2f} %".format(train_loss, train_accuracy))

  validate_batch_loss = 0.0
  validate_correct_predictions = 0
  validate_samples = 0
  model_3.eval()
  with torch.no_grad():
    for xvalidate, yvalidate in validate_loader:

      optimizer.zero_grad()

      xvalidate = xvalidate.to(device)
      yvalidate = yvalidate.to(device)
      validate_prob = model_3(xvalidate)

      #calculating batch accuracy
      _, validate_prediction = torch.max(validate_prob, 1)
      validate_samples += yvalidate.size(0)
      validate_correct_predictions += (validate_prediction == yvalidate).sum().item()

      loss = loss_function(validate_prob, yvalidate)

      #calculating batch loss
      validate_batch_loss += loss.item()

  # calculating validate loss for epoch
  validate_loss = validate_batch_loss / len(validate_loader)
  validate_losses.append(validate_loss)

  # calculating validate accuracy for epoch
  validate_accuracy = 100 * (validate_correct_predictions / validate_samples)
  validate_accuracies.append(validate_accuracy)
  print("validation loss: {:.4f}  validation accuracy: {:.2f} %".format(validate_loss, validate_accuracy))

end_time = time.time()
epoch_time = end_time - start_time

print("Wall Clock Time for One Epoch Vgg16:", epoch_time, "seconds")

epoch number ---> 1 
training loss: 13.4855  training accuracy: 33.62 %
validation loss: 1.2808  validation accuracy: 33.67 %
Wall Clock Time for One Epoch Vgg16: 33.34986734390259 seconds
